In [1]:
!python -V

Python 3.7.3


In [97]:
import urllib
import json
from datetime import datetime
from datetime import date
from pandas.io.json import json_normalize
import pandas as pd


def download_weather_climate_division(date_to_start: str, date_to_end: str, state: str):
    base_url = "http://data.rcc-acis.org/GridData"
    start_date = datetime.strptime(date_to_start, '%Y%m%d').strftime("%Y%m%d")
    end_date = datetime.strptime(date_to_end, '%Y%m%d').strftime("%Y%m%d")
        
    params_mint = {"state":"IL",
                     "sdate":start_date,
                     "edate":end_date,
                     "grid":21,
                     "elems":[{"name":"mint","area_reduce":"county_mean"}]}
    
    data = json.dumps(params_mint).encode('utf-8')
    headers = {"Content-Type": "application/json"}
    
    try:
        req = urllib.request.Request(base_url, data, headers)
        with urllib.request.urlopen(req) as response:
            mint_json = response.read()
    except Exception as e:
        print(e)

    return mint_json

In [99]:
dump = download_weather_climate_division('20190916', '20190925', 'IL')
dump2 = download_weather_climate_division('20190901','20190912', 'IL')

In [103]:
json_string=dump.decode("utf-8")
data = json.loads(json_string)
basic_data = json_normalize(data, 'data').set_index(0)
df = json_normalize(basic_data[1])
df['date'] = basic_data.index
df = df.set_index('date')
df = df[['17031', '17043']]

json_string2 = dump2.decode('utf-8')
data2 = json.loads(json_string2)
normed_data2 = json_normalize(data2, 'data').set_index(0)
df2 = json_normalize(normed_data2[1])
df2['date'] = normed_data2.index
df2 = df2.set_index('date')
df2 = df2[['17031', '17043']]

In [107]:
final = pd.concat([df, df2])
pd.DataFrame.sort_index(final)

,17031,17043
date,,
2019-09-01,60.920860,59.967575
2019-09-02,60.349689,59.517746
2019-09-03,64.677284,62.736107
2019-09-04,60.677906,58.523918
2019-09-05,55.992039,53.767521
2019-09-06,58.930668,57.054020
2019-09-07,55.987076,53.646538
2019-09-08,60.085693,56.892323
2019-09-09,60.614933,59.882767


In [ ]:
# request data
params_n = json.dumps({"state":"IL",
                     "sdate":start_date,
                     "edate":end_date,
                     "grid":21,
                     "elems":[{"name":"mint","area_reduce":"climdiv_mean"}]})

req_n = urllib3.request(config['prism_url'],params_n,{"Content-Type":"application/json"})
response = urllib3.urlopen(req_n)
n_json = json.loads(response.read())['data']


params_x = json.dumps({"state":"IL",
                     "sdate":start_date,
                     "edate":end_date,
                     "grid":21,
                     "elems":[{"name":"maxt","area_reduce":"climdiv_mean"}]})
req_x = urllib3.request(config['prism_url'],params_x,{"Content-Type":"application/json"})
response = urllib3.urlopen(req_x)
x_json = json.loads(response.read())['data']

params_p = json.dumps({"state":"IL",
                     "sdate":start_date,
                     "edate":end_date,
                     "grid":21,
                     "elems":[{"name":"pcpn","area_reduce":"climdiv_mean"}]})

req_p = urllib3.request(config['prism_url'],params_p,{"Content-Type":"application/json"})
response = urllib3.urlopen(req_p)
p_json = json.loads(response.read())['data']

In [5]:
for i in range(1,10):
    file = open("../output_files/110"+str(i),"w+")
    for n,x,p in zip(n_json,x_json,p_json):
        ymd = n[0].split("-")
        if bool(n[1]) and bool(x[1]) and bool(p[1]):
            file.write("IL" + "\t" + str(i) + "\t" + ymd[0] + "\t"  + ymd[1].lstrip("0") + "\t" + ymd[2].lstrip("0") + "\t" + str(float("{0:.1f}".format(n[1]["IL0"+str(i)]))) + "\t" + str(float("{0:.1f}".format(x[1]["IL0"+str(i)]))) + "\t" + str(float("{0:.2f}".format(p[1]["IL0"+str(i)]))) + "\n")
    file.close()

/home/jallen17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


TypeError: 'module' object is not callable